In [2]:
import numpy as np
import random
from collections import defaultdict

In [2]:
class env:
    def __init__(self, map_version, step_limit, debug = 0, use_seed = False, seed = 42):
        
        self.seed = seed
        self.use_seed = use_seed
        
        if self.use_seed:
            random.seed(self.seed)
            

        self.debug = debug
        self.size = size
        self.base_step_limit = step_limit
        self.step_limit = self.base_step_limit
        #self.valid_moves = ['up', 'down', 'left', 'right', 'pass']
        self.valid_moves = [0, 1, 2, 3, 4]
        self.move_map = {0:[-1,0], 
                         1:[1,0], 
                         2:[0,-1], 
                         3:[0,1], 
                         4:[0,0]}
        self.initialize_map()
        self.initialize_resources()
        

    
    def initialize_map_1(self):
        self.map = np.zeros((16, 16), dtype=int)
        
        
    def initialize_resources(self):
        self.resources = []
        index = [i for i in range(self.size**2) if (i != 0) and (i != self.size-1)]
        all_indexes = random.sample(index, self.number_of_resources*self.amount_of_resource)
        res_ind = []
        for i in range(self.number_of_resources):
            resource = random.sample(all_indexes, self.amount_of_resource)
            all_indexes = [r for r in all_indexes if r not in resource]
            resource = [(int(r/self.size),r%self.size) for r in resource]
            empty_map = np.zeros((self.size, self.size), dtype=int)
            for ind in range(len(resource)):
                x, y = resource[ind][0], resource[ind][1]                
                self.map[x, y] = i+1
                empty_map[x, y] = 1
            self.resources.append(empty_map)
        goal_index = [i for i in range(self.number_of_resources)]
        goals = random.sample(goal_index, 2)
        self.agents_goals = goals
        self.agents_res_collected = [[], []]
        self.agents_self_goal_collected = [0, 0]
        self.agents_othr_goal_collected = [0, 0]
        self.agents_nthr_goal_collected = [0, 0]
                
    def step(self, agent_numb, move_code):
        if self.step_limit == 0:
            if self.debug:
                print('step limit reached, error')
        else:
            self.step_limit -= 1
            if self.debug:
                print(self.step_limit, 'steps left')
        if move_code not in self.valid_moves:
            if self.debug:
                print('icorrect move')
        else:
            agent_old_coords = self.agents_coords[agent_numb]
            move = self.move_map[move_code]
            agent_new_coords = l3 = [l+r for l,r in zip(agent_old_coords, move)]
            correct_move = True
            for coord in agent_new_coords:
                if (coord < 0) or (coord >= self.size):
                    if self.debug:
                        print('icorrect move')
                    correct_move = False
            other_coords = self.agents_coords[1-agent_numb]
            if agent_new_coords == other_coords:
                if self.debug:
                    print('icorrect move')
                correct_move = False
            if correct_move:
                self.map[agent_old_coords[0],agent_old_coords[1]] = 0
                self.map[agent_new_coords[0],agent_new_coords[1]] = -1 - agent_numb
                self.agents_coords[agent_numb] = agent_new_coords
                for res in range(self.number_of_resources):
                    if self.resources[res][agent_new_coords[0],agent_new_coords[1]] != 0:
                        self.agents_res_collected[agent_numb].append(res)
                        self.resources[res][agent_new_coords[0],agent_new_coords[1]] = 0
                        if res == self.agents_goals[agent_numb]:
                            self.agents_self_goal_collected[agent_numb]+=1
                            if self.debug:
                                print('collected self goal resource:', res)
                        elif res == self.agents_goals[1 - agent_numb]:
                            self.agents_othr_goal_collected[agent_numb]+=1
                            if self.debug:
                                print('collected other goal resource:', res)
                        else:
                            self.agents_nthr_goal_collected[agent_numb]+=1
                            if self.debug:
                                print('collected neither goal resource:', res)
                            
    def observation(self, agent_numb, with_map = 0):
        feature_vec = []
        self_other_coords = [self.agents_coords[agent_numb], 
                             self.agents_coords[1 - agent_numb] 
                            ]
        for agnt_coords in self_other_coords:
            agent_map = np.zeros((self.size, self.size), dtype=int)
            agent_map[agnt_coords[0],agnt_coords[1]] = 1
            agent_map = np.reshape(agent_map, self.size*self.size)
            feature_vec.extend(agent_map)
            
        for res in range(len(self.resources)):
            feature_vec.extend(np.reshape(self.resources[res], self.size*self.size))
        
        if with_map:
            feature_vec.extend(np.reshape(self.map, self.size*self.size))
        
        return feature_vec
    
    def reward(self):
        
        n_self_c_self = self.agents_self_goal_collected[0]
        n_othr_c_self = self.agents_othr_goal_collected[1]
        
        n_self_c_othr = self.agents_othr_goal_collected[0]
        n_othr_c_othr = self.agents_self_goal_collected[1]
        
        n_self_c_nthr = self.agents_nthr_goal_collected[0]
        n_othr_c_nthr = self.agents_nthr_goal_collected[1]
        
        reward = (n_self_c_self + n_othr_c_self)**2 + \
                 (n_self_c_othr + n_othr_c_othr)**2 - \
                 (n_self_c_nthr + n_othr_c_nthr)**2 
        
        return reward

    def reset(self):
        if self.use_seed:
            random.seed(self.seed)
        self.step_limit = self.base_step_limit
        self.initialize_map()
        self.initialize_resources()

In [3]:
buffer_ag_1 = []
buffer_ag_2 = []

def sample_from_buffer(buffer, batch_size=1):
    batch = random.sample(buffer,batch_size)
    inputs = [i[0] for i in batch]
    actions = [i[1] for i in batch]
    revards = [i[2] for i in batch]
    
    inputs = torch.FloatTensor(inputs)
    actions = torch.FloatTensor(actions)
    revards = torch.FloatTensor(revards)
    
    return inputs, actions, revards

In [ ]:
def create_map():
    

In [ ]:
# 107, 110, 115

In [48]:
env_1 = env(10, 20, seed = 137)
env_1.map

array([[-1,  0,  1,  0,  0,  0,  0,  0,  0, -2],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  2,  3,  0,  0,  0],
       [ 0,  3,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  2,  1,  3,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  2,  0,  0,  2,  0,  3],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [41]:
env_1.reset()

In [42]:
env_1.agents_goals

[0, 2]

In [43]:
env_1.map

array([[-1,  0,  0,  0,  0,  0,  1, -2],
       [ 0,  0,  0,  0,  3,  0,  0,  1],
       [ 0,  0,  1,  0,  0,  2,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 3,  0,  0,  0,  2,  0,  0,  0],
       [ 0,  2,  0,  0,  0,  2,  0,  3],
       [ 0,  0,  0,  0,  0,  0,  0,  3]])

In [445]:
#self.valid_moves = ['up', 'down', 'left', 'right', 'pass']
#self.valid_moves = [0, 1, 2, 3, 4]

In [7]:
map_ = np.zeros((18, 18), dtype=int)

In [74]:
map_size = 16
def create_empty_map_v1(map_size):
    if isinstance(map_size, int):
        if map_size >=6:
            # create empty map with aaditional place for external walls
            map_obj = np.zeros((map_size+2, map_size+2), dtype=int)
            # add external walls
            map_obj[0], map_obj[-1] = 1, 1
            for i in range(map_size+2):
                map_obj[i][0], map_obj[i][-1] = 1, 1

            # add room, with side of half of the map size
            room_size = map_size//2
            map_obj[-(room_size+1)][-(room_size+1):] = 1
            for j in range(room_size):
                map_obj[-(j+1)][-(room_size+1)]=1

            # add doors in both walls of the room
            map_obj[-(room_size+1)][-2:-1] = 0
            map_obj[-(room_size-1)][-(room_size+1)] = 0
            return(map_obj)
        elif map_size < 6 and map_size >= 0:
            print('Error: map size is too small, correct size is 6+')
    else:
        print('Error: incorrect map size type: expected int')

In [76]:
map_1 = create_empty_map_v1(16)
print(map_1)

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1]
 [1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


True

In [29]:
29//2

14